In [2]:
"""
Working version of stitching algorithm 
Where it correctly mates the pieces

Things to add during this iteration: 
1) Translates the labels so don't have to recompute
2) where doesn't delete faces to end
3) Fixes the normals



Changes made: 
1) Made transfer of labels instead of recomputing
2) Made sure the old facet generator doesn't have overlapping arrays
"""

"\nWorking version of stitching algorithm \nWhere it correctly mates the pieces\n\nThings to add during this iteration: \n1) Translates the labels so don't have to recompute\n2) where doesn't delete faces to end\n3) Fixes the normals\n\n\n\nChanges made: \n1) Made transfer of labels instead of recomputing\n2) Made sure the old facet generator doesn't have overlapping arrays\n"

In [3]:
import trimesh
import numpy as np
from collections import Counter
import time
import math
from tqdm import tqdm

In [4]:
#pick the mesh to process: 
file_location = "./test_meshes/"
file_location = "./"
file_name ="seperated_mesh.off"

In [5]:
#visualize the neuron to make sure it is correct
visualize = False
if visualize == True:
    visualize_mesh = trimesh.load_mesh(file_location + file_name )
    visualize_mesh.show()

# Gets parts that are outside of the main mesh

In [6]:
def filter_mesh_significant_outside_pieces(unfiltered_mesh,significance_threshold=2000,n_sample_points=1000):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces, including the main one that are not inside of main mesh

    """

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []

    #find piece with largest size
    max_index = 0
    max_face_len = len(significant_pieces[max_index].faces)

    for i in range(1,len(significant_pieces)):
        if max_face_len < len(significant_pieces[i].faces):
            max_index = i
            max_face_len = len(significant_pieces[i].faces)

    print("max_index = " + str(max_index))
    print("max_face_len = " + str(max_face_len))

    final_mesh_pieces = []

    main_mesh = significant_pieces[max_index]

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        if i != max_index:
            #get a random sample of points
            # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
            idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
            points = mesh.vertices[idx,:]
            
            
            start_time = time.time()
            signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
            print(f"Total time = {time.time() - start_time}")

            outside_percentage = sum(signed_distance < 0)/n_sample_points
            if outside_percentage > 0.9:
                final_mesh_pieces.append(mesh)
                print(f"Mesh piece {i} OUTSIDE mesh")
            else:
                print(f"Mesh piece {i} inside mesh :( ")
                
    return main_mesh,final_mesh_pieces



In [7]:
#Runs the filtering function for inside and outside meshes
global_timer = time.time()

unfiltered_mesh = trimesh.load_mesh(file_location + file_name)

#setting thresholds
significance_threshold=30 #number of faces needed for pieces to be considered to be kept
n_sample_points = 3 #number of points sampled on the mesh for determination of inside or outside
start_time = time.time()

#the main mesh is the first mesh in the piece
main_mesh,child_meshes = filter_mesh_significant_outside_pieces(unfiltered_mesh,
                            significance_threshold=significance_threshold,
                                n_sample_points=n_sample_points)
print(f"Total time for Mesh Cleansing: {time.time() - start_time}")


There were 3902 pieces after mesh split
There were 378 pieces found after size threshold
max_index = 0
max_face_len = 1845336
Total time = 26.88869071006775
Mesh piece 1 OUTSIDE mesh
Total time = 0.120361328125
Mesh piece 2 inside mesh :( 
Total time = 0.32430410385131836
Mesh piece 3 OUTSIDE mesh
Total time = 1.434826374053955
Mesh piece 4 OUTSIDE mesh
Total time = 0.4488375186920166
Mesh piece 5 inside mesh :( 
Total time = 0.34423160552978516
Mesh piece 6 inside mesh :( 
Total time = 0.3300507068634033
Mesh piece 7 inside mesh :( 
Total time = 0.3342270851135254
Mesh piece 8 inside mesh :( 
Total time = 0.351229190826416
Mesh piece 9 inside mesh :( 
Total time = 0.3279838562011719
Mesh piece 10 inside mesh :( 
Total time = 0.3731999397277832
Mesh piece 11 inside mesh :( 
Total time = 0.6318912506103516
Mesh piece 12 inside mesh :( 
Total time = 0.6762332916259766
Mesh piece 13 inside mesh :( 
Total time = 0.5420873165130615
Mesh piece 14 inside mesh :( 
Total time = 0.63698053359985

# FACET GENERATION AND FILTERING FUNCTIONS

In [8]:
import math
def area(vertices):
    """
    Calculates the area of a 3D triangle from it's coordinates
    """
    side_a = np.linalg.norm(vertices[0]-vertices[1])
    side_b = np.linalg.norm(vertices[1]-vertices[2])
    side_c = np.linalg.norm(vertices[2]-vertices[0])
    s = 0.5 * ( side_a + side_b + side_c)
    return math.sqrt(s * (s - side_a) * (s - side_b) * (s - side_c))

def find_polygon_area(mesh,list_of_faces):
    "Calculates the area of a 3D polygon that is created from connected traingles"
    return(sum([area(mesh.vertices[mesh.faces[r]]) for r in list_of_faces]))


In [9]:
# need a way of finding the neighbors that have to share adjacent faces
def create_neighbors_lookup(mesh):
    start_time = time.time()
    neighbors_lookup = dict([(i,[]) for i in range(0,len(mesh.faces))])
    #print(f"Creating empty dictionary : {time.time() - start_time}")
    
    start_time = time.time()
    for adj in mesh.face_adjacency:
        neighbors_lookup[adj[0]].append(adj[1])
        neighbors_lookup[adj[1]].append(adj[0])
    #print(f"Filling in neighbors lookup : {time.time() - start_time}")
    
    return neighbors_lookup

# need a way of finding the neighbors that have to share adjacent faces
def create_facet_lookup(mesh):
    start_time = time.time()
    facets_lookup = np.ones(len(main_mesh.faces))*-1
    
    start_time = time.time()
    for jj,facet in enumerate(mesh.facets):
        for fac in facet:
            facets_lookup[fac] = jj
    
    print(f"Filling in facet lookup : {time.time() - start_time}")
    
    return facets_lookup

def filter_and_expand_facets(new_mesh,
                             first_pass_size_threshold=2000,
                             normal_closeness=0.985):
    
    """
    
    """
    
    #filter the facets that are below a size threshold
    new_facets = new_mesh.facets[np.where(new_mesh.facets_area > first_pass_size_threshold)[0]]

    #is a fast lookup for the neighbors of a certain face
    neighbors_lookup = create_neighbors_lookup(new_mesh)
    
    #generate a normals lookup table and see if works faster than regular lookup
    normal_lookup = {}

    for i in range(0,len(new_mesh.faces)):
        normal_lookup[i] = new_mesh.face_normals[i]
    
    global_start_time = time.time()
    final_facets= [0]*len(new_facets)
    
    #creates entire list of neighbors that have been added in expansion
    #if a neighbor was already added that was already in the current facet
    #then that facet has already been absorbed and need to delete that
    

    #these are facets that need to be deleted at the end
    
    
    
    """
    Other way to optimize

    1) Create lookup dictionary that maps the face back to the facet
    2) when add a neighbor add the set of all their facets to the facet to delete


    """
    print("Total number of facets = " + str(len(new_facets)))
    facet_lookup = create_facet_lookup(new_mesh)
    facets_to_delete = set()
    
    for i,facet in enumerate(new_facets):
#         mini_global_start_time = time.time()
    #     print(facet)
    #     print(type(facet))
    #     start_time = time.time()
    #     print([find_neighbors(k) for k in facet])
    
    

#         new_time = time.time()
#         # make sure that there are no repeats of facets by checking for common elements
#         if not set(total_added_neighbors).isdisjoint(facet):
#             facets_to_delete.append(i)
#             #print("found overlapping neighbor")
#             continue
#         print(f"Time for check = {time.time() - new_time}")
            
        if i in facets_to_delete:
            continue
    
    
    #gets all of the neighbors
        total_neighbors = list(set(np.hstack([neighbors_lookup[k] for k in facet])).difference(set(facet)))
    #     print(total_neighbors)
    #     print(f"time initial neighbors: {(time.time() - start_time)}")
        neighbors_to_add = []
        neighbors_checked = []
        #print(total_neighbors)

        #just get the normal from one of the faces already in the facet
    #     start_time = time.time()
        facet_normal = normal_lookup[facet[0]]

    #     total_dot_time = 0
        while len(total_neighbors) > 0:
            current_neighbor = total_neighbors.pop()
            neighbors_checked.append(current_neighbor)

    #         print("--------------")
    # #         #check to see if neighbor has same normal face
    # #         start_dot_time = time.time()
    # #         dot_result = np.dot(new_mesh.face_normals[current_neighbor],facet_normal) #> normal_closeness
    # #         print(dot_result)
    # #         print((time.time() - start_dot_time) * 1000)

    #         start_dot_time = time.time()
    #         print(current_neighbor)
    #         #a = new_mesh.face_normals[current_neighbor]

    #         print((time.time() - start_dot_time) * 1000)
    #         dot_result = a[0]*facet_normal[0] + a[1]*facet_normal[1] + a[2]*facet_normal[2]  > normal_closeness
    #         print(dot_result)
    #         print((time.time() - start_dot_time) * 1000)
    #         print("--------------")

    #         total_dot_time += (time.time() - start_dot_time)*1000

            a = normal_lookup[current_neighbor]
            if a[0]*facet_normal[0] + a[1]*facet_normal[1] + a[2]*facet_normal[2]  > normal_closeness:

                neighbors_to_add.append(current_neighbor)
                #get the neighbors of this current face
                for neigh in neighbors_lookup[current_neighbor]:
                    #only add those neighbors that havent already been checked, in original facet group, or already in list to check
                    if neigh not in neighbors_checked and neigh not in facet and neigh not in total_neighbors:
                        total_neighbors.append(neigh)
    #     print(f"Total dot time: {total_dot_time}")
    #     print(f"time loop: {(time.time() - start_time)}")
    #     print("neighbors_to_add = " + str(neighbors_to_add))abs
    #     print("neighbors_checked = " + str(neighbors_checked))
    #     print("adding list = " + str(list(facet) + neighbors_to_add))
    #     start_time = time.time()
        
        #get all the facets of the neighbors to add
        neighbors_to_add_facets = facet_lookup[neighbors_to_add]
        facets_to_delete = facets_to_delete.union(set(neighbors_to_add_facets))
        
        final_facets[i] = np.array(list(facet) + neighbors_to_add)
    #     print(f"Appending to list: {(time.time() - start_time)}")

#         print(f"Total time: {(time.time() - mini_global_start_time)}")
    #     print("------------------------------------------------------")


    print(f"Total Facet building time: {(time.time() - global_start_time)}")
    print(f"Found Total number of overlapping faces: {len(facets_to_delete)}")
    
    #print("facets_to_delete = " + str(facets_to_delete))
    #delete the faces = 
    final_facets = np.delete(final_facets,list(facets_to_delete),axis=0)
    return final_facets

def filter_final_facets(gap_mesh):
    """
    Gets the facets faces list and the center points of these facets from mesh
    Filters:
    1) Only lets facets greater than first_pass_size_threshold exist
      **** might need to look at this because area is that of facets before expansion
    2) Has to have a high convex border
    
    Expansions:
    1) Expands the facet group to neighbors that are within the normal_closeness
    for their normals when doing the dot product
    
    Order:
    1) Size filtering
    2) Expansion
    3) Convex Border filtering
    """
    
    final_facets = filter_and_expand_facets(gap_mesh,
                                 first_pass_size_threshold=2000,
                                 normal_closeness=0.985)
    
    

    # after computing final faces, filter for convexity
    edges = gap_mesh.edges_sorted.reshape((-1, 6)) #groups all the edges belonging to the corresponding face in one row
    final_facets_mean = np.zeros(len(final_facets))
    
    
    #make lookup table for face number to spot in the adjacency edges
    face_adjacency_index_lookup = [[] for i in gap_mesh.faces]
    for i,faces in enumerate(gap_mesh.face_adjacency):
        for f in faces:
            face_adjacency_index_lookup[f].append(i)


    for j,facet in enumerate(final_facets):
        # get the edges for each facet
        edges_facet = [edges[i].reshape((-1, 2)) for i in [facet]][0] #stores all the edges belonging to that face

        #get the indexes of the boundary edges:
        indexes = trimesh.grouping.group_rows(edges_facet, require_count=1)
        edge_0 = edges_facet[indexes]

        #find the faces that correspond to the boundary edges
        edge_0_faces = [facet[int(k/3)] for k in indexes]


        #2) Find the indexes of the edges int he face_adajacency_edges and store the projections
        adjacency_values = []
        for edge,edge_face in zip(edge_0,edge_0_faces):
            possible_adj_indexes = face_adjacency_index_lookup[edge_face]

            for index in possible_adj_indexes:
                if len(set(edge).intersection(set(gap_mesh.face_adjacency_edges[index]))) >= 2:
                    #print(f"adj edge = {e} and boundary edge = {edge}")
                    adjacency_values.append(gap_mesh.face_adjacency_angles[index]) # the metric we actually want to measure
                    break


        final_facets_mean[j] = np.mean(adjacency_values)
        
        

    #filter the final facets and output them so they can be plotted
    adjacency_threshold =  0.8
    final_facets_mean_filtered = np.array(final_facets)[final_facets_mean > adjacency_threshold]

    #print(final_facets_mean_filtered)
    #Compute the centers
    final_facets_centers = []
    
    for filt in final_facets_mean_filtered: 
        #print("filt = " + str(filt))
        unique_vertices = gap_mesh.vertices[np.unique(gap_mesh.faces[filt].ravel())].astype("float")
        final_facets_centers.append((np.mean(unique_vertices[:,0]),
                          np.mean(unique_vertices[:,1]),
                          np.mean(unique_vertices[:,2])))
    
    return final_facets_mean_filtered,final_facets_centers


# DEBUGGING THE FILTER FACETS SPEED

In [10]:
# start_time = time.time()

# #facets_list, centers_list,neighbors_added = filter_final_facets(main_mesh)
# final_facets,facet_centers = filter_final_facets(main_mesh)

# print(f"Total time for overlapping check = {time.time() - start_time}")

# final_facets_1 = final_facets[0]
# len(total_added_neighbors)


# len(final_facets)

# new_time = time.time()
# # make sure that there are no repeats of facets by checking for common elements
# if not set(total_added_neighbors).isdisjoint(final_facets_1):
#     #facets_to_delete.append(i)
#     print("found overlapping neighbor")
# print(f"Time for check = {(time.time() - new_time)*len(final_facets)/60}")

# """
# Conclusion this method would be too slow
# """

# FAILED WAY TO OPTIMIZE #1

In [11]:
# start_time = time.time()
# for ff1,facet1 in enumerate(facets_list):
#     #get the difference of both of them
#     start_time = time.time()
#     for ff2,facet2 in enumerate(facets_list):
#         if ff2>ff1 and (ff2 not in facets_to_delete):
#             common_elements = set(facet1).intersection(set(facet2))
#             #print(common_elements)
#             if (len(common_elements)/len(facet1)) >= 0.75:
#                 print("Found overlapping faces")
#                 facets_to_delete.append(ff2)
#     print(f"Total time for overlapping check = {(time.time() - start_time)*len(facets_list)/60}")
#     break

In [12]:
# THE OLDER FACET FILTERING METHOD THAT DOESN'T RETURN AS MANY FACETS BUT MISSES OUT ON THE 
#SMALL ONES

def filter_final_facets_working(gap_mesh,final_facets):
    """
    Gets the facets faces list and the center points of these facets from mesh
    Filters:
    1) Only lets facets greater than first_pass_size_threshold exist
      **** might need to look at this because area is that of facets before expansion
    2) Has to have a high convex border
    
    Expansions:
    1) Expands the facet group to neighbors that are within the normal_closeness
    for their normals when doing the dot product
    
    Order:
    1) Size filtering
    2) Expansion
    3) Convex Border filtering
    """


    # after computing final faces, filter for convexity
    edges = gap_mesh.edges_sorted.reshape((-1, 6)) #groups all the edges belonging to the corresponding face in one row
    final_facets_mean = np.zeros(len(final_facets))
    
    
    #make lookup table for face number to spot in the adjacency edges
    face_adjacency_index_lookup = [[] for i in gap_mesh.faces]
    for i,faces in enumerate(gap_mesh.face_adjacency):
        for f in faces:
            face_adjacency_index_lookup[f].append(i)


    for j,facet in enumerate(final_facets):
        # get the edges for each facet
        edges_facet = [edges[i].reshape((-1, 2)) for i in [facet]][0] #stores all the edges belonging to that face

        #get the indexes of the boundary edges:
        indexes = trimesh.grouping.group_rows(edges_facet, require_count=1)
        edge_0 = edges_facet[indexes]

        #find the faces that correspond to the boundary edges
        edge_0_faces = [facet[int(k/3)] for k in indexes]


        #2) Find the indexes of the edges int he face_adajacency_edges and store the projections
        adjacency_values = []
        for edge,edge_face in zip(edge_0,edge_0_faces):
            possible_adj_indexes = face_adjacency_index_lookup[edge_face]

            for index in possible_adj_indexes:
                if len(set(edge).intersection(set(gap_mesh.face_adjacency_edges[index]))) >= 2:
                    #print(f"adj edge = {e} and boundary edge = {edge}")
                    adjacency_values.append(gap_mesh.face_adjacency_angles[index]) # the metric we actually want to measure
                    break


        final_facets_mean[j] = np.mean(adjacency_values)
        
        

    #filter the final facets and output them so they can be plotted
    adjacency_threshold =  0.8
    final_facets_mean_filtered = np.array(final_facets)[final_facets_mean > adjacency_threshold]

    #Compute the centers
    final_facets_centers = []
    
    for filt in final_facets_mean_filtered: 
        #print("filt = " + str(filt))
        unique_vertices = gap_mesh.vertices[np.unique(gap_mesh.faces[filt].ravel())].astype("float")
        final_facets_centers.append((np.mean(unique_vertices[:,0]),
                          np.mean(unique_vertices[:,1]),
                          np.mean(unique_vertices[:,2])))
    
    return final_facets_mean_filtered,final_facets_centers

In [13]:
def filter_final_facets_optimized_with_checks(example_mesh,
                                  normal_closeness=0.99,
                                  first_pass_size_threshold=6000):
    """
    Way of computing facets that uses trimesh grouping function and normals threshold
    instead of the pre-built trimesh.facets
    
    """

    #get the facets for the child mesh
    start_time_total = time.time()

    #switch out with dot product
    
    
    #Old way of computing the dot product manually
    total_normals = example_mesh.face_normals[example_mesh.face_adjacency]
    
    #using manual method
#     print(len(total_normals[:,0]*total_normals[:,1]))
    start_normal = time.time()
    total_normal_dots = np.sum(total_normals[:,0]*total_normals[:,1],axis=1)
    print(f"Total normals took {time.time() - start_normal}")
    ''' DONT HAVE TO DO DOT PRODUCT BECAUSE JUST WANT TO MULTIPLY THE ROWS OF EACH COLUMN'''
#     print("About to do dot product")
#     dot_start = time.time()
#     a = total_normals[:,0]
#     b = total_normals[:,1].T
#     #new way using the dot product
#     total_normal_dots = np.dot(a,b)
    
    
    #get the True/False value if face adjacency is within normal closeness
    start_normal = time.time()
    total_normal_dots_facet_mask = total_normal_dots > normal_closeness
    print(f"Boolean mask finished: {time.time() - start_normal}")

    #getting the grouping of the faces into facets within the threshold
    start_normal = time.time()
    components = trimesh.graph.connected_components(example_mesh.face_adjacency[total_normal_dots_facet_mask],
                                          nodes=np.arange(len(example_mesh.faces)),
                                          min_len=2,
                                          engine="None")
    print(f"Grouping took: {time.time() - start_normal}")
    
    

    #print(f"Lenght of facets BEFORE filtering = {len(components)}")
    
    #filter by the size
    
    start_normal = time.time()
    size_filtered_components = [facet_list for facet_list in components 
                                    if find_polygon_area(example_mesh,facet_list) > first_pass_size_threshold]
    print(f"Filtering edges by size finished: {time.time() - start_normal}")
    


    #filter by the convexity
    #final_facets_mean_filtered,final_facets_centers = filter_final_facets(example_mesh,size_filtered_components)
    
    start_normal = time.time()
    final_facets_mean_filtered,final_facets_centers = filter_final_facets_working(example_mesh,size_filtered_components)
    print(f"Filtering by convexity and getting centers took: {time.time() - start_normal}")

    
    print(f"Total Time = {time.time() - start_time_total}")

    return final_facets_mean_filtered,final_facets_centers
    

# ACTUALLY CALCULATING THE FACETS

In [14]:
"""
Rule: 
1) if mesh size is above a threshold --> run the unoptimized less careful version
2) if mesh is below that size --> run optimized more generous (but with all the filters )

"""
facet_time = time.time()
start_time = time.time()

length_threshold = 60000

if len(main_mesh.faces > length_threshold):
    print(f" face length {len(main_mesh.faces)} using unoptimized facets")
    main_mesh_facets,main_mesh_facets_centers = filter_final_facets(main_mesh)
    
else:
    print(f" face length {len(main_mesh.faces)} using OPTIMIZED facets")
    main_mesh_facets,main_mesh_facets_centers = filter_final_facets_optimized_with_checks(main_mesh)
    

print(f"Finished { len(main_mesh_facets)} facets for main mesh: {time.time() - start_time}")
child_meshes_facets= []
for jj,gap_mesh in enumerate(child_meshes):
    
    start_time = time.time()
    if len(gap_mesh.faces) > length_threshold:
        print(f" face length {len(gap_mesh.faces)} using unoptimized facets")
        child_meshes_facets.append(filter_final_facets(gap_mesh))
    else:
        print(f" face length {len(gap_mesh.faces)} using OPTIMIZED facets")
        child_meshes_facets.append(filter_final_facets_optimized_with_checks(gap_mesh))

    
    print(f"Finished facets for child {jj} : {time.time() - start_time} with facet length = {len(child_meshes_facets[jj][0])}")


print(f"Total time for facets: {time.time() - facet_time}")
#child_meshes_facets = [filter_final_facets(gap_mesh) for gap_mesh in child_meshes]

 face length 1845336 using unoptimized facets
Total number of facets = 59411
Filling in facet lookup : 0.11599278450012207
Total Facet building time: 16.507033586502075
Found Total number of overlapping faces: 17861


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:162: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:162: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:162: FutureWarning: in the future negative indices will not be ignored by `numpy.delete`.


Finished 11118 facets for main mesh: 55.251808881759644
 face length 117018 using unoptimized facets
Total number of facets = 3693
Filling in facet lookup : 0.007793903350830078
Total Facet building time: 0.3681912422180176
Found Total number of overlapping faces: 1091
Finished facets for child 0 : 2.4274303913116455 with facet length = 699
 face length 16458 using OPTIMIZED facets
Total normals took 0.000545501708984375
Boolean mask finished: 0.0001308917999267578
Grouping took: 0.0061550140380859375
Filtering edges by size finished: 0.444427490234375
Filtering by convexity and getting centers took: 0.6628556251525879
Total Time = 1.1227242946624756
Finished facets for child 1 : 1.1232471466064453 with facet length = 644
 face length 32560 using OPTIMIZED facets
Total normals took 0.0010762214660644531
Boolean mask finished: 0.00014495849609375
Grouping took: 0.011479616165161133
Filtering edges by size finished: 0.7998263835906982
Filtering by convexity and getting centers took: 1.12

# Check that facet extraction for each piece worked

In [16]:
# # download the facets and make sure they match

# current_facets = main_mesh_facets
# current_mesh = main_mesh
# index = "main"


# facets_group = np.zeros(len(current_mesh.faces)).astype(int)

# for i,facet_group in enumerate(current_facets):
#     for face in facet_group:
#         facets_group[face] = i + 1 #so that you reserve the label 0 for blenders none

# np.savez("./test_meshes/" + str(index) + "_debug_june_facet_" + str(index) + ".npz",
#          facets_group=facets_group)

# from print_trimesh import print_trimesh
# print_trimesh(current_mesh,"./test_meshes/piece_debug_june_" + str(index) + ".off")




In [17]:
# # download the facets and make sure they match
# final_facets_mean_filtered

# # download the facets list and see if the process worked:
# index = 1
# facets_group = np.zeros(len(child_meshes[index].faces)).astype(int)

# for i,facet_group in enumerate(size_filtered_components):
#     for face in facet_group:
#         facets_group[face] = i + 1 #so that you reserve the label 0 for blenders none

# np.savez("./test_meshes/" + "1_facet_piece_" + str(index) + ".npz",
#          facets_group=facets_group,facets_group_centers=final_facets_centers)

# from print_trimesh import print_trimesh
# print_trimesh(child_meshes[index],"./test_meshes/piece_" + str(index) + ".off")


# len(final_facets_mean_filtered)

# START ITERATIVE PROCESS THAT CONNECTS THE MESHES

In [18]:

def apply_bbox_filter(child,min_bb_zone,max_bb_zone):
    """
    Determines if child is withing the bounding box zone
    designated by the bounding box corners
    
    """
    #get the min and max of the bounding box for the mesh
    min_bb = np.array(child.bounding_box.vertices).min(0)
    max_bb = np.array(child.bounding_box.vertices).max(0)
    
    #print(min_bb,max_bb)
    #print(min_bb_zone,max_bb_zone)
    
    #if fails any of these checks then return false, else return True
    if min(min_bb[0],max_bb[0])>max_bb_zone[0]:
        print("returning x greater max")
        return False
    
    if max(min_bb[0],max_bb[0])<min_bb_zone[0]:
        print("returning x less min")
        return False
    
    if min(min_bb[1],max_bb[1])>max_bb_zone[1]:
        print("returning y greater max")
        return False
    
    if max(min_bb[1],max_bb[1])<min_bb_zone[1]:
        print("returning y less min")
        return False
        
    if min(min_bb[2],max_bb[2])>max_bb_zone[2]:
        print("returning z greater max")
        return False
    
    if max(min_bb[2],max_bb[2])<min_bb_zone[2]:
        print("returning z less mim")
        return False
    return True

import math
def area(vertices):
    """
    Calculates the area of a 3D triangle from it's coordinates
    """
    side_a = np.linalg.norm(vertices[0]-vertices[1])
    side_b = np.linalg.norm(vertices[1]-vertices[2])
    side_c = np.linalg.norm(vertices[2]-vertices[0])
    s = 0.5 * ( side_a + side_b + side_c)
    return math.sqrt(s * (s - side_a) * (s - side_b) * (s - side_c))

def find_polygon_area(mesh,list_of_faces):
    "Calculates the area of a 3D polygon that is created from connected traingles"
    return(sum([area(mesh.vertices[mesh.faces[r]]) for r in list_of_faces]))

In [19]:
# restitching functions
import trimesh
import numpy as np
from collections import Counter
import time
import math

#gets the projection of point p onto line a
def ClosestPointOnLine(a, b, p):
    ap = p-a
    ab = b-a
    #base_vector = ab
    result = np.dot(ap,ab)/np.dot(ab,ab) # * ab
    return result




#now have the mesh and the facet faces, can send to function
def stitch_mesh_piece_vp3(new_mesh,facet_1,facet_2,
                          delete_facets=False,
                         return_added_mesh = False,
                         fix_normals = False):
    """
    Changed since last version: 
    1) parameter for deleting facets at end or not
    2) parameter for returning added mesh or not 
    3) Changed normals check to print statement and not exception
    
    
    """
    #how to find the normals of facet groups:
    facet_group_1_normal = new_mesh.face_normals[facet_1[0]]
    facet_group_2_normal = new_mesh.face_normals[facet_2[0]]


    #get the correct version of the normals: (might need to flip them if going in opposite direction)
    if np.dot(facet_group_1_normal,facet_group_2_normal) > 0.8:
        print("same direction normals")
        pass
    elif np.dot(facet_group_1_normal,facet_group_2_normal) < -0.8:
        print("opposite direction normals")
        facet_group_2_normal = facet_group_2_normal*-1
    else:
        print("Not correct normals")
        #raise Exception("Not correct normals")

    #print(facet_group_1_normal,facet_group_2_normal)

    # make each row correspond to a single face 
    edges = new_mesh.edges_sorted.reshape((-1, 6))
    # get the edges for each facet
    edges_facet = [edges[i].reshape((-1, 2)) for i in [facet_1,facet_2]]
    edges_boundary = np.array([i[trimesh.grouping.group_rows(i, require_count=1)]
                               for i in edges_facet])

    #the list of boundary edges and unique points in the boundary edges
    edge_0 = edges_boundary[0]
    edge_1 = edges_boundary[1]

    #gets the unique number of points
    edge_0_points = np.unique(np.hstack(edge_0))
    edge_1_points = np.unique(np.hstack(edge_1))
    print("Found boundary edges")
    """
    get the dot product of all the points
    """

    #get any 2 points on the triangle and make that the reference edge
    edge_0_anchor_points = new_mesh.vertices[[edge_0_points[0],edge_0_points[1]]]

    #gets the starting index for the 1st facet (so that the start of the stitching is close)
    max_index = 0
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

    for i in range(1,len(edge_0_points)):
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_0_points[i]])

        if current_magnitude > max_magnitude:
            max_index = i
            max_magnitude = current_magnitude

    edge_0_starting_point = edge_0_points[max_index]

    #gets the starting index for the 2nd facet (so that the start of the stitching is close)
    max_index = 0
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

    for i in range(1,len(edge_1_points)):
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_1_points[i]])
        if current_magnitude > max_magnitude:
            max_index = i
            max_magnitude = current_magnitude

    edge_1_starting_point = edge_1_points[max_index]

    print(f"starting edge 1st facet = {edge_0_starting_point}, starting edge 2nd facet= {edge_1_starting_point}, ")
    #print(new_mesh.vertices[edge_0_starting_point],new_mesh.vertices[edge_1_starting_point])

    """
    Need to order the points for restitching

    Pseudocode: 
    1) Get starting piont
    2) Find the two edges corresponding to that point
    3) Need to decide which direction to start....
    - go in direction that make the cross of the (1st and last) point in the same direction of the 
    normal of the first facet
    4) loop through and record the orders of the vertices as you traverse along the edges 
    until you arrive back at the start
    5) Error if:
        a. You arrive back at the start and haven't processed all the edges
        b. Processsed all the edges but haven't arrived back at start

    6) Repeat steps 1 through 5 for 2nd facet group
    """
    start_point_list = [edge_0_starting_point,edge_1_starting_point]
    edge_list = [edge_0,edge_1]
    edge_order_list = []

    #loop that organizes the unique boundary points into the correct order
    for i,start_point in enumerate(start_point_list):
        print(f"Starting Organizing vertices for side {i}")
        #print(f"start_point = {start_point}")
        edge_order = [start_point]
        processed_edges = []

        #find the matching edges
        starting_edges_indices = np.where(np.logical_or(edge_list[i][:,0] == start_point,edge_list[i][:,1] == start_point) == True)[0]
        print(starting_edges_indices)
        starting_edges = edge_list[i][starting_edges_indices]
        #print(f"starting edges = {starting_edges}") #the list of the two possible edges

        if starting_edges.size < 4:
            raise Exception("Not enough edges for 1st facet start point")

        if starting_edges.size > 4:
            raise Exception("Too many edges for 1st facet start point") 

        #np.where(starting_edges[1,:] != start_point)[0][0]
        #gets the vectors that will be used for the cross product
        #print("np.where(starting_edges[0,:] != start_point)[0][0] = " + str(np.where(starting_edges[0,:] != start_point)[0][0]))
        #print("np.where(starting_edges[1,:] != start_point)[0][0] = " + str(np.where(starting_edges[1,:] != start_point)[0][0]))


        #gets the possible starting vectors
        possible_starting_vector_1 = new_mesh.vertices[starting_edges[0,:][np.where(starting_edges[0,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]
        possible_starting_vector_2 = new_mesh.vertices[starting_edges[1,:][np.where(starting_edges[1,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]
        

        #find the cross product of the starting vectors
        starting_edges_cross = np.cross(possible_starting_vector_1,possible_starting_vector_2)

        #make sure that the order of the vectors goes so that the cross product is in line with the starting normal
        #this ensures the the circular direction of the stitching will be the same
        if np.dot(starting_edges_cross,facet_group_1_normal) > 0:
            print("Edge 1 picked for direction")
            processed_edges.append(starting_edges_indices[0])
            current_vertex = starting_edges[0][np.where(starting_edges[0,:] != start_point)[0][0]]
        else:
            print("Edge 2 picked for direction")
            processed_edges.append(starting_edges_indices[1])
            #print("np.where(starting_edges[1,:] != start_point) = " + str(np.where(starting_edges[1,:] != start_point)))
            current_vertex = starting_edges[1][np.where(starting_edges[1,:] != start_point)[0][0]]

        #print(f"current_vertex = {current_vertex}" )
        #print("edge_list = " + str(edge_list))

        #now iterate through number of 
        for z in range(1,edge_list[i][:,0].size):
            #print("edge_order_temp = " + str(edge_order))
            if current_vertex == start_point:
                print("Start vertex reached before processed all of edges")
                
                """
                
                These should be ok because the extra loops are created from holes inside and this process should always get the outside loop
                
                
                """
                break

            #get the next edge
            counter = 0
            next_vertex = -1
            for j,edg in enumerate(edge_list[i]):
                #print("edg = " + str(edg))
                #print("processed_edges = " + str(processed_edges))
                if current_vertex in edg and j not in processed_edges:
                    current_edge_index = j
                    if edg[0] != current_vertex:
                        next_vertex = edg[0]
                    else:
                        next_vertex = edg[1]


                    counter += 1
                    if counter >= 2:
                        #raise Exception(f"More than 2 edges possibilities for {current_vertex}")
                        #Don't want to make it an exception anymore put just print out warning
                        print("More than 2 edges possibilities for {current_vertex}") # BAC change

            #make sure the next vertex was found
            if next_vertex <= -1:
                raise Exception(f"No next vertex was found for {current_vertex} ")

            #if found next vertex then add the old vertex and edge index
            #to the processed edges lists and the order of vertices
            processed_edges.append(current_edge_index)
            edge_order.append(current_vertex)

            current_vertex = next_vertex


        edge_order_list.append(edge_order)
        print(f"edge_{i}_order done, len = {len(edge_order)} ")#"= {edge_order}")

    lengths_of_boundaries = [len(x) for x in edge_order_list]
    bigger = lengths_of_boundaries.index(max(lengths_of_boundaries))
    smaller = 1-bigger

    #print(f"index of bigger facets = {bigger}\nindex of smaller facets = {smaller}",)

    #calculates the number of vertices will be stitched to each vertices on smaller side
    dividend = int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])
    remainder = lengths_of_boundaries[bigger] - int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])*lengths_of_boundaries[smaller]

    print(f"dividend = {dividend}, remainder = {remainder}")

    #loop that adds the new faces
    print("About to add faces")
    start_time = time.time()
    new_faces = []
    current_bigger = 0
    for i,current_smaller in enumerate(edge_order_list[smaller]):
        #print("current_smaller =" + str(current_smaller))
        #print("current_bigger=" + str(current_bigger))
        if i == 0:
            new_faces.append([current_smaller,edge_order_list[smaller][-1],edge_order_list[bigger][current_bigger]])
        else:
            new_faces.append([current_smaller,edge_order_list[smaller][i-1],edge_order_list[bigger][current_bigger]])

        for j in range(0,dividend + int(i<remainder)):
            if current_bigger > len(edge_order_list[bigger]):
                raise Exception("Somehow rapped around too much")

            if current_bigger >= len(edge_order_list[bigger])-1:
                next_bigger = 0
            else:
                next_bigger = current_bigger+1

            new_faces.append([current_smaller,edge_order_list[bigger][current_bigger],
                                edge_order_list[bigger][next_bigger]])

            current_bigger += 1

    #print("new_faces = " + str(new_faces))
    print(f"Finished adding faces: {time.time() - start_time}")
    

    print("Starting creating stitch mesh")
    start_time = time.time()
    stitch_mesh = trimesh.Trimesh()

    stitch_mesh.vertices = new_mesh.vertices
    stitch_mesh.faces = np.vstack([new_mesh.faces, new_faces])
    print(f"Finished creating stitch mesh: {time.time() - start_time}")
    



    if delete_facets == True:
        #now take away the original facet faces:
        total_faces = np.linspace(0,len(stitch_mesh.faces)-1,len(stitch_mesh.faces)).astype("int")
        facet_faces = np.hstack([facet_1 ,facet_2])
        faces_to_keep = set(total_faces).difference(set(facet_faces))
        faces_to_keep

        stitch_mesh = stitch_mesh.submesh([list(faces_to_keep)])[0]

    if fix_normals == True:
        trimesh.repair.fix_inversion(stitch_mesh)
        trimesh.repair.fix_winding(stitch_mesh)
        trimesh.repair.fix_normals(stitch_mesh)
    
    #print("Finished stitching")

    if return_added_mesh == True:
        added_mesh = trimesh.Trimesh()
        added_mesh.vertices = new_mesh.vertices
        added_mesh.faces = new_faces
        trimesh.repair.fix_inversion(added_mesh)
        trimesh.repair.fix_winding(added_mesh)
        trimesh.repair.fix_normals(added_mesh)

        return stitch_mesh,added_mesh
    
    else:
        return stitch_mesh

In [20]:
# restitching functions
import trimesh
import numpy as np
from collections import Counter
import time
import math

#gets the projection of point p onto line a
def ClosestPointOnLine(a, b, p):
    ap = p-a
    ab = b-a
    #base_vector = ab
    result = np.dot(ap,ab)/np.dot(ab,ab) # * ab
    return result



#now have the mesh and the facet faces, can send to function
def stitch_mesh_piece_vp4(new_mesh,facet_1,facet_2,
                          delete_facets=False,
                         return_added_mesh = False,
                         fix_normals = False):

    """
    Changed since last version: 
    1) parameter for deleting facets at end or not
    2) parameter for returning added mesh or not 
    3) Changed normals check to print statement and not exception


    """
    #how to find the normals of facet groups:
    facet_group_1_normal = new_mesh.face_normals[facet_1[0]] #main_mesh_normals
    facet_group_2_normal = new_mesh.face_normals[facet_2[0]] #child_mesh_normals


    #get the correct version of the normals: (might need to flip them if going in opposite direction)
    if np.dot(facet_group_1_normal,facet_group_2_normal) > 0.8:
        raise Exception("same direction normals")
    elif np.dot(facet_group_1_normal,facet_group_2_normal) < -0.8:
        print("opposite direction normals")
    else:
        print("Not correct normals")
        #raise Exception("Not correct normals")

    # make each row correspond to a single face 
    edges = new_mesh.edges_sorted.reshape((-1, 6))
    # get the edges for each facet
    edges_facet = [edges[i].reshape((-1, 2)) for i in [facet_1,facet_2]]
    edges_boundary = np.array([i[trimesh.grouping.group_rows(i, require_count=1)]
                               for i in edges_facet])

    #the list of boundary edges and unique points in the boundary edges
    edge_0 = edges_boundary[0]
    edge_1 = edges_boundary[1]

    #gets the unique number of points
    edge_0_points = np.unique(np.hstack(edge_0))
    edge_1_points = np.unique(np.hstack(edge_1))
    print("Found boundary edges")
    """
    get the dot product of all the points
    """

    #get any 2 points on the triangle and make that the reference edge
    edge_0_anchor_points = new_mesh.vertices[[edge_0_points[0],edge_0_points[1]]]

    #gets the starting index for the 1st facet (so that the start of the stitching is close)
    max_index = 0
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

    for i in range(1,len(edge_0_points)):
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_0_points[i]])

        if current_magnitude > max_magnitude:
            max_index = i
            max_magnitude = current_magnitude

    edge_0_starting_point = edge_0_points[max_index]

    #gets the starting index for the 2nd facet (so that the start of the stitching is close)
    max_index = 0
    max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

    for i in range(1,len(edge_1_points)):
        current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_1_points[i]])
        if current_magnitude > max_magnitude:
            max_index = i
            max_magnitude = current_magnitude

    edge_1_starting_point = edge_1_points[max_index]

    print(f"starting edge 1st facet = {edge_0_starting_point}, starting edge 2nd facet= {edge_1_starting_point}, ")
    #print(new_mesh.vertices[edge_0_starting_point],new_mesh.vertices[edge_1_starting_point])

    """
    Need to order the points for restitching

    Pseudocode: 
    1) Get starting piont
    2) Find the two edges corresponding to that point
    3) Need to decide which direction to start....
    - go in direction that make the cross of the (1st and last) point in the same direction of the 
    normal of the first facet
    4) loop through and record the orders of the vertices as you traverse along the edges 
    until you arrive back at the start
    5) Error if:
        a. You arrive back at the start and haven't processed all the edges
        b. Processsed all the edges but haven't arrived back at start

    6) Repeat steps 1 through 5 for 2nd facet group
    """
    start_point_list = [edge_0_starting_point,edge_1_starting_point]
    edge_list = [edge_0,edge_1]
    edge_order_list = []



    #loop that organizes the unique boundary points into the correct order
    for i,start_point in enumerate(start_point_list):
        print(f"Starting Organizing vertices for side {i}")
        #print(f"start_point = {start_point}")
        edge_order = [start_point]
        processed_edges = []

        #find the matching edges to the starting point
        starting_edges_indices = np.where(np.logical_or(edge_list[i][:,0] == start_point,edge_list[i][:,1] == start_point) == True)[0]

        starting_edges = edge_list[i][starting_edges_indices]
        #print(f"starting edges = {starting_edges}") #the list of the two possible edges

        if starting_edges.size < 4:
            raise Exception("Not enough edges for 1st facet start point")

        if starting_edges.size > 4:
            raise Exception("Too many edges for 1st facet start point") 

        #np.where(starting_edges[1,:] != start_point)[0][0]
        #gets the vectors that will be used for the cross product
        #print("np.where(starting_edges[0,:] != start_point)[0][0] = " + str(np.where(starting_edges[0,:] != start_point)[0][0]))
        #print("np.where(starting_edges[1,:] != start_point)[0][0] = " + str(np.where(starting_edges[1,:] != start_point)[0][0]))

        """*************** where pick the starting edge starts ************
        The way it works: 
        1) Gets the two possible starting edges
        2) Generates the vectors for the edges where origin is the starting point
        3) Gets the cross porduct of both vectors
        4) Chooses the cross product that is in the direction of the face normals

        Why that doesn't work:
        1) they are opposite normals


        """

        """
        Possible other solution:
        1) Get the starting points on the child edge
        2) Pick the first edge as the default edge

        """
        processed_edges.append(starting_edges_indices[0])
        current_vertex = starting_edges[0][np.where(starting_edges[0,:] != start_point)[0][0]]




    #         #gets the possible starting vectors from the two possible edges
    #         possible_starting_vector_1 = new_mesh.vertices[starting_edges[0,:][np.where(starting_edges[0,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]
    #         #just start with a random edge
    #         possible_starting_vector_2 = new_mesh.vertices[starting_edges[1,:][np.where(starting_edges[1,:] != start_point)[0][0]]] - new_mesh.vertices[start_point]


    #         #find the cross product of the starting vectors
    #         starting_edges_cross = np.cross(possible_starting_vector_1,possible_starting_vector_2)

    #         #make sure that the order of the vectors goes so that the cross product is in line with the starting normal
    #         #this ensures the the circular direction of the stitching will be the same
    #         if np.dot(starting_edges_cross,facet_group_1_normal) > 0:
    #             print("Edge 1 picked for direction")
    #             processed_edges.append(starting_edges_indices[0])
    #             current_vertex = starting_edges[0][np.where(starting_edges[0,:] != start_point)[0][0]]
    #         else:
    #             print("Edge 2 picked for direction")
    #             processed_edges.append(starting_edges_indices[1])
    #             #print("np.where(starting_edges[1,:] != start_point) = " + str(np.where(starting_edges[1,:] != start_point)))
    #             current_vertex = starting_edges[1][np.where(starting_edges[1,:] != start_point)[0][0]]

        #print(f"current_vertex = {current_vertex}" )
        #print("edge_list = " + str(edge_list))



        """*************** where pick the starting edge ends ************"""
        #now iterate through number of 
        for z in range(1,edge_list[i][:,0].size):
            #print("edge_order_temp = " + str(edge_order))
            if current_vertex == start_point:
                print("Start vertex reached before processed all of edges")

                """

                These should be ok because the extra loops are created from holes inside and this process should always get the outside loop


                """
                break

            #get the next edge
            counter = 0
            next_vertex = -1
            for j,edg in enumerate(edge_list[i]):
                #print("edg = " + str(edg))
                #print("processed_edges = " + str(processed_edges))
                if current_vertex in edg and j not in processed_edges:
                    current_edge_index = j
                    if edg[0] != current_vertex:
                        next_vertex = edg[0]
                    else:
                        next_vertex = edg[1]


                    counter += 1
                    if counter >= 2:
                        #raise Exception(f"More than 2 edges possibilities for {current_vertex}")
                        #Don't want to make it an exception anymore put just print out warning
                        print("More than 2 edges possibilities for {current_vertex}") # BAC change

            #make sure the next vertex was found
            if next_vertex <= -1:
                raise Exception(f"No next vertex was found for {current_vertex} ")

            #if found next vertex then add the old vertex and edge index
            #to the processed edges lists and the order of vertices
            processed_edges.append(current_edge_index)
            edge_order.append(current_vertex)

            current_vertex = next_vertex


        edge_order_list.append(edge_order)
        print(f"edge_{i}_order done, len = {len(edge_order)} ")#"= {edge_order}")

    #     #print the edge orders
    #     for e in edge_order_list:
    #         print(type(e))
    lengths_of_boundaries = [len(x) for x in edge_order_list]



    """ ************ PROCESS OF ORDERING THE EDGE PATHS SO THAT *********
    main mesh loop goes in counter clockwise in reference to the gap
    child goes clockwise in reference to gap




    """

    """
    1) Pick the starting point as your P point
    2) For each point in ordered list calculate point - P and store that vector
    3) Do the cross product of list[0:n-2] x list[1:n-1]
    4) Take the sum of these lists of cross products
    5) Do the dot product to compare the direction with the normal vector 
    Result: 
    - if positive --> conuter-clockwise according to normal
    - if negative --> clockwise according to normal

    """

    starter_point = np.array([0,0,0])

    #     list_of_points = [1,2,3,4,0]

    #     list_of_points = [list_of_points[len(list_of_points) - x -1] for x in range(0,len(list_of_points))]
    #     print(list_of_points)

    #print("facet_group_2_normal = " + str(facet_group_2_normal))

    for ed,e_loop in enumerate(edge_order_list):

        #get the vertices according to the points
        vertices_list = new_mesh.vertices[e_loop]

        #get the cross product of the offsetted list
        #vertices_list[0:len(vertices_list)-1,:]
        """ Wrong earlier previous way

        cross_products = np.cross(vertices_list[0:len(vertices_list)-1,:],vertices_list[1:len(vertices_list)])
        """

        cross_products = np.cross(vertices_list[0:len(vertices_list),:],
                             np.vstack([vertices_list[1:len(vertices_list),:],vertices_list[0,:]]))

        sum_cross_products = np.sum(cross_products,axis=0)

        #print("cross_products = " + str(cross_products))
        #print("sum_cross_products = " + str(sum_cross_products))

        #print("Before edge list = " + str(edge_order_list[ed]))
        if ed == 0:
            #get the dot product
            normals_dot = np.dot(sum_cross_products,facet_group_1_normal)
            #print(' normals_dot = ' + str(normals_dot))
            if normals_dot > 0:
                print("Main originally was counter-clockwise --> keeping")


            else:
                print("Main originally was clockwise --> flipping")
                edge_order_list[ed] = [e_loop[len(e_loop) - x -1] for x in range(0,len(e_loop))]


        else: 
            # for the children want the cross product to be counter clockwise
            normals_dot = np.dot(sum_cross_products,facet_group_2_normal)
            #print(' normals_dot = ' + str(normals_dot))
            if normals_dot > 0:
                print("Child originally was counter-clockwise --> flipping")
                edge_order_list[ed] = [e_loop[len(e_loop) - x -1] for x in range(0,len(e_loop))]
            else:
                print("Child originally was clockwise --> keeping")


        #print("After edge list = " + str(edge_order_list[ed]))
    """  SHOWS THAT DOING THE CROSS PRODUCT THAT WAY WORKS
    #do the cross products manually
    for jj in range(0,len(vertices_list)-1):
        print(np.cross(vertices_list[jj],vertices_list[jj+1]))

    """




    #getting which one is the bigger one
    bigger = lengths_of_boundaries.index(max(lengths_of_boundaries))
    smaller = 1-bigger

    if bigger == 0:
        starter_face = "child" #if the bigger one was the main, then child is the smaller one you start from
    else:
        starter_face = "main" #if the bigger one was the child, then main is the smaller one you start from


    print("smaller_face = " + str(starter_face))    

    """ The rules that have to be following in order for normals to be correctly aligned
    1) if the smaller is the child (will be traveling in clockwise direction
    --> need to stitch points as:
    Regular: other_2, other_1,current_point
    Neighbor: current,other_1,current-1


    1) if the smaller is the main mesh (
    --> need to stitch points as:
    Regular: current_point,other_1,other_2
    Neighbor: current,current-1,other
    """


    #print(f"index of bigger facets = {bigger}\nindex of smaller facets = {smaller}",)

    #calculates the number of vertices will be stitched to each vertices on smaller side
    dividend = int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])
    remainder = lengths_of_boundaries[bigger] - int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])*lengths_of_boundaries[smaller]

    print(f"dividend = {dividend}, remainder = {remainder}")

    #loop that adds the new faces
    print("About to add faces")
    start_time = time.time()
    new_faces = []
    current_bigger = 0
    total_counter= 0
    end_number = 1000
    for i,current_smaller in enumerate(edge_order_list[smaller]):
        #print("current_smaller =" + str(current_smaller))
        #print("current_bigger=" + str(edge_order_list[bigger][current_bigger]))

        #connecting to the neighbor on the shorter side
        """
        if i == 0:

            new_faces.append([current_smaller,edge_order_list[smaller][-1],edge_order_list[bigger][current_bigger]])
        else:
            new_faces.append([current_smaller,edge_order_list[smaller][i-1],edge_order_list[bigger][current_bigger]])
        """


        if starter_face == "main":
            new_faces.append([current_smaller,edge_order_list[bigger][current_bigger],edge_order_list[smaller][i-1]])
        else:
            new_faces.append([current_smaller,edge_order_list[smaller][i-1],edge_order_list[bigger][current_bigger]])

        total_counter += 1
        if total_counter > end_number:
            break
        for j in range(0,dividend + int(i<remainder)):
            if current_bigger > len(edge_order_list[bigger]):
                raise Exception("Somehow rapped around too much")

            if current_bigger >= len(edge_order_list[bigger])-1:
                next_bigger = 0
            else:
                next_bigger = current_bigger+1

            if starter_face == "main":
                new_faces.append([edge_order_list[bigger][next_bigger],
                                  edge_order_list[bigger][current_bigger],
                                  current_smaller,
                                ])
            else:
                new_faces.append([
                                current_smaller,
                                  edge_order_list[bigger][current_bigger],
                                  edge_order_list[bigger][next_bigger],

                                ])

            current_bigger += 1

            total_counter += 1
            if total_counter > end_number:
                break
        if total_counter > end_number:
                break


    #print("new_faces = " + str(new_faces))
    print(f"Finished adding faces: {time.time() - start_time}")


    print("Starting creating stitch mesh")
    start_time = time.time()
    stitch_mesh = trimesh.Trimesh()

    stitch_mesh.vertices = new_mesh.vertices
    stitch_mesh.faces = np.vstack([new_mesh.faces, new_faces])
    print(f"Finished creating stitch mesh: {time.time() - start_time}")




    if delete_facets == True:
        #now take away the original facet faces:
        total_faces = np.linspace(0,len(stitch_mesh.faces)-1,len(stitch_mesh.faces)).astype("int")
        facet_faces = np.hstack([facet_1 ,facet_2])
        faces_to_keep = set(total_faces).difference(set(facet_faces))
        faces_to_keep

        stitch_mesh = stitch_mesh.submesh([list(faces_to_keep)])[0]

    if fix_normals == True:
        trimesh.repair.fix_inversion(stitch_mesh)
        trimesh.repair.fix_winding(stitch_mesh)
        trimesh.repair.fix_normals(stitch_mesh)

    #print("Finished stitching")

    if return_added_mesh == True:
        added_mesh = trimesh.Trimesh()
        added_mesh.vertices = new_mesh.vertices
        added_mesh.faces = new_faces
        trimesh.repair.fix_inversion(added_mesh)
        trimesh.repair.fix_winding(added_mesh)
        trimesh.repair.fix_normals(added_mesh)

        return stitch_mesh,added_mesh

    else:
        return stitch_mesh

In [21]:
#makes a copy of the main mesh and the main mesh facets so can just run this cell again if errors:
main_mesh_saved = main_mesh.copy()
main_mesh_facets_centers_saved = main_mesh_facets_centers.copy()
main_mesh_facets_saved = main_mesh_facets.copy()

#save off the child information as well
from copy import deepcopy

child_meshes_saved = deepcopy(child_meshes)
child_meshes_facets_saved = deepcopy(child_meshes_facets) #has both the centers and the facet lists


In [24]:
restitch_time = time.time()

In [40]:
"""

Pseudocode for loop at the end that will keep everything going:
1) When process a child, will add that index to a list
2) Have no_new_children_processed counter set at end of loop 
    if no children were added to main mesh
    --> this will prompt the expansion of the initial parameters
3) If this gets too high



Things that still need to add:
1) Better way of making sure that the normals are good
- Can sample one of the neighboring points and flip normals if the dot product is negative


Change list: 
1) Reduced the stitch distance
2) Added the copy features that allows this cell to be rerun without rerunning whole notebook
3) added the new tie_breaker for childs that want to connect to same parent is just the one with closest facet
    Uses the added feature of child_meshes_stitch_distances that is saved along the way
4) Iterates through the repeated faces instead of just doing so once which was incorrect before
5) Fixed bug that was adding to current_main_mesh but then was using main mesh also in the loop
6) Made changes to the stitching mechanism that not error if find a vertices with more than 2 edges
--> because did observe some facets with cut out faces along the boundary
7) Changed the consider_same_direction normal as False 
8) Changed the max index error that was used for finding the starting point in stitch meshes

"""
total_stitch_processing_time = time.time()
#load our main mesh stuff from the copies
#makes a copy of the main mesh and the main mesh facets so can just run this cell again if errors:
main_mesh = main_mesh_saved.copy()
main_mesh_facets_centers = main_mesh_facets_centers_saved.copy()
main_mesh_facets = main_mesh_facets_saved.copy()

#load the child info from the saved copies
child_meshes_facets_saved = deepcopy(child_meshes_facets_saved)
child_meshes  = deepcopy(child_meshes_saved)


#sets 
initial_parameters = dict(bounding_box_threshold=4000,
                         stitch_distance_threshold=1200,
                         size_ratio_threshold=0.15,
                         normal_closeness= 0.95)

bounding_box_threshold = initial_parameters["bounding_box_threshold"]
stitch_distance_threshold = initial_parameters["stitch_distance_threshold"]
size_ratio_threshold = initial_parameters["size_ratio_threshold"]
normal_closeness=initial_parameters["normal_closeness"]

no_new_children_multiplier = 0
bbox_expansion_percentage = 0.10
stitch_expansion_percentage = 0.20
size_ratio_expansion_percentage = 0.10

no_new_children_limit = 4

consider_same_direction_normals = False
children_processed = []

#lists to store the faces that need to be removed later from main mesh
child_faces_to_remove = []
main_faces_to_remove = []

while len(children_processed) < len(child_meshes):
    stitch_loop_time = time.time()
    if no_new_children_multiplier >= no_new_children_limit:
        print("The number of times expanding the thresholds has exceed the limit /n Just returning main mesh")
        print(f"total_stitch_processing_time = {time.time() - total_stitch_processing_time}")
        raise Exception("Finished the mesh stitiching")
    
    #update the thresholds
    bounding_box_threshold = initial_parameters["bounding_box_threshold"]*(1 + bbox_expansion_percentage*no_new_children_multiplier)
    stitch_distance_threshold = initial_parameters["stitch_distance_threshold"]*(1 + stitch_expansion_percentage*no_new_children_multiplier)
    #reduces the 
    size_ratio_threshold = initial_parameters["size_ratio_threshold"]*(1 - size_ratio_expansion_percentage*no_new_children_multiplier)
    
    #print thresholds
#     print(f"bounding_box_threshold = {bounding_box_threshold}")
#     print(f"stitch_distance_threshold = {stitch_distance_threshold}" )
#     print(f"size_ratio_threshold = {size_ratio_threshold}")

    #get the main mesh facets normals
    main_mesh_normals = [main_mesh.face_normals[fac[0]] for fac in main_mesh_facets]
    hit_indexes_list= []
    
    
    #dictionary to save the stitch points
    child_meshes_stitch_facets = dict()
    child_meshes_stitch_face_ratios = dict()
    child_meshes_stitch_distances = dict()
    for i,child in enumerate(child_meshes):
        
        if i in children_processed:
            #print(f"Child {i} already processed")
            continue
        
        print(f"Starting Child {i}")

        #initialize the stitch index
        #child_meshes_stitch_facets[i] = [-1,-1]

        #two highest points for the bounding box
        min_bb = np.array(main_mesh.bounding_box.vertices).min(0)
        max_bb = np.array(main_mesh.bounding_box.vertices).max(0)

        min_bb_zone = min_bb - bounding_box_threshold
        max_bb_zone = max_bb + bounding_box_threshold



        #then send mesh to function that decides if with
        pass_bbox_filter = apply_bbox_filter(child,min_bb_zone,max_bb_zone)

        if not pass_bbox_filter:
            print("skipped by bounding box filter")
            continue

        #### used for validation to make sure that bounding box was correctly filtering
    #     #add the following to main mesh just to visualize
    #     final_mesh = final_mesh + child
    #     print("just added")

        #
        """

        Do pairwise calculation of distances between centers of facets on child piece and main piece
            Filter the pairs for stitch distance threshold
            Filter the pairs for matching or opposite normals
            Filter for those that are reasonable in size matching (within 50% of each other)

        If there are still ties after this point then pitch the best matching size

        If None remaining then keep going (might loosen up the parameters later), BUT WITH GLOBAL CHECK SO DOESN’T KEEP ITERATING


        """
 
        """ OLD WAY OF COMPUTING PAIRWISE ***************************************************************
        
        child_facets,child_facets_centers = child_meshes_facets[i]

        facet_distances = np.zeros((len(child_facets_centers),len(main_mesh_facets_centers)))
        facet_normals = np.zeros((len(child_facets_centers),len(main_mesh_facets_centers)))

        start_time = time.time()
        #do the pairwise comparison
        for cc,child_center in enumerate(child_facets_centers):

            #get the normal of the child
            child_normal = child.face_normals[child_facets[cc][0]]
            for mm, main_center in enumerate(main_mesh_facets_centers):
                #get the euclidean distance
    #             facet_distances[cc,mm] = math.sqrt((child_center[0] - main_center[0])**2 + 
    #                                                  (child_center[1] - main_center[1])**2 + 
    #                                                  (child_center[2] - main_center[2])**2 )



                facet_distances[cc,mm] = np.linalg.norm(np.array(child_center) - np.array(main_center))

                main_normal = main_mesh_normals[mm]
    #             print("main_normal = " + str(main_normal))
    #             print("child_normal = " + str(child_normal))
                #get whether the normals are line up (or possibly opposite)
                normals_dot = np.dot(child_normal,main_normal)
                #print(normals_dot)
                if consider_same_direction_normals == True:
                    facet_normals[cc,mm] =  normals_dot < -0.95  or normals_dot > 0.95
                else:
                    facet_normals[cc,mm] =  normals_dot < -0.95 # or normals_dot > 0.95


        
        print(f"Total pairwise : {time.time() - start_time}")

    #     if i == 1:
    #         4,12
    #         print("4,12 facet distance = " + str(facet_distances[4,12]))

        #apply the filters
        #stitch_distance_booleans = facet_distances < stitch_distance_threshold

        
        #how to debug certain faces that have been filtered
#         main_facet_bebug = 124
#         piece_facet_debug = 11
        
#         if i == 2 and (0 in children_processed):
#             print("**********" + str(facet_normals[piece_facet_debug,main_facet_bebug]) + "**********")
#             print("**********" + str((facet_distances < stitch_distance_threshold)[piece_facet_debug,main_facet_bebug]) + "**********")
        
        
        #get the indexes that make it through the first two filters
        hit_indexes = np.where(np.logical_and(facet_distances < stitch_distance_threshold,facet_normals)== True) 

        possible_stitch_pairs = np.vstack(hit_indexes).T
        
        #print("possible_stitch_pairs = " + str(possible_stitch_pairs))
        
        face_0_unique_facets = np.unique(hit_indexes[0])
        face_1_unique_facets = np.unique(hit_indexes[1])
        
        if len(hit_indexes[0]) <= 0:
            print(f"Child {i} There was no possible stitch found after stitch distance and face normal filters")
            continue

        if len(hit_indexes[0]) > 0:
            face_0_facet_sizes = dict([(u,find_polygon_area(child,child_facets[u])) for u in face_0_unique_facets])
            face_1_facet_sizes = dict([(u,find_polygon_area(main_mesh,main_mesh_facets[u])) for u in face_1_unique_facets])

            

         *************************************************************** """
        
        """  NEW WAY OF COMPUTING THE PAIRWISE CALCULATIONS  *************************************************************** """
        
        child_facets,child_facets_centers = child_meshes_facets[i]
        pairs_start_time = time.time()

        
        if len(child_facets_centers) == 0:
            print("child_facets_centers for child {i} was 0, so skipping")
            continue
        
        start_time = time.time()
        a_New = np.array(child_facets_centers).astype("float")
        b_New = np.array(main_mesh_facets_centers).astype("float")

        print(len(a_New),len(b_New))
#         print("starting distance matrix")
        start_time = time.time()
        from scipy.spatial import distance_matrix
        a_b_distance = distance_matrix(a_New, b_New)
        
        #print(f"Time = {time.time() - start_time}")

        #now get the indexes that are within stitch distance

#         print("min(a_b_distance) = " + str(np.amin(a_b_distance.shape)))
        indexes_not = np.where(a_b_distance < stitch_distance_threshold)
        print(f"Done distance matrix: {time.time() - start_time}")
        
        
#         print("(indexes_not) = " + str((indexes_not)))
#         print("starting normals")
        start_time = time.time()

#         #old way to do it
#         child_normals = child.face_normals[indexes_not[0]]
#         main_normals = main_mesh.face_normals[indexes_not[1]]

        if not indexes_not[0].any():
            print(f"Child {i} There were no points close enough")
            continue

        #old way to do it
#         print(indexes_not[0])
#         print(child_facets[indexes_not[0]])
 
        
        child_normals = child.face_normals[[p[0] for p in child_facets[indexes_not[0]]]]
        main_normals = main_mesh.face_normals[[p[0] for p in main_mesh_facets[indexes_not[1]]]]
                                           
                                           
                                           
#         print(child_facets[indexes_not[0]][0])
#         print('child_normals = ' + str(child_normals))
#         print(main_mesh_facets[indexes_not[1]][0])
#         print('main_normals = ' + str(main_normals))
        #start_time = time.time()
        #dot_products = np.dot(child_normals,main_normals.T)
        
        total_normal_dots = np.sum(child_normals*main_normals,axis=1)
#         print("total_normal_dots = " + str(total_normal_dots))
        total_normal_dots_facet_mask = total_normal_dots < -normal_closeness
        
#         print(f"Done Normals generation: {time.time() - start_time}")
#         print("starting pair generation")
        
        
        start_time = time.time()
        final_pairs = (np.array(indexes_not).T)[total_normal_dots_facet_mask]
        #print(final_pairs)
        
        
        #get the sizes of all the unique ones
        face_0_unique_facets = np.unique(final_pairs[:,0])
        face_1_unique_facets = np.unique(final_pairs[:,1])
        #print("final_pairs = " + str(final_pairs))
#         print("face_0_unique_facets = " + str(face_0_unique_facets))
#         print("face_1_unique_facets = " + str(face_1_unique_facets))
        
        
        if len(final_pairs) <= 0:
            print(f"Child {i} There was no possible stitch found after stitch distance and face normal filters")
            continue

        if len(final_pairs[0]) > 0:
            face_0_facet_sizes = dict([(u,find_polygon_area(child,child_facets[u])) for u in face_0_unique_facets])
            face_1_facet_sizes = dict([(u,find_polygon_area(main_mesh,main_mesh_facets[u])) for u in face_1_unique_facets])

        
        possible_stitch_pairs = final_pairs
        print(f"Done Generating final pairs: {time.time() - pairs_start_time}")
        
        
        """  NEW WAY OF COMPUTING THE PAIRWISE  *************************************************************** """
        




    #     print("possible_stitch_pairs = " + str(possible_stitch_pairs))
    #     print("len(hit_indexes) = " + str(len(hit_indexes)))
    #     print("hit_indexes[0].any() = " + str(hit_indexes[0].any()))
    #     print("hit_indexes = " + str(hit_indexes))
    #     print("hit_indexes[0] = " + str(hit_indexes[0]))
    #     print("len(hit_indexes[0]) = " + str(len(hit_indexes[0])))

        #find the sizes of all of them
        face_pair_sizes = np.zeros(len(possible_stitch_pairs[:,0]))
        face_size_ratios = np.zeros(len(possible_stitch_pairs[:,0]))

        #print("possible_stitch_pairs = " + str(possible_stitch_pairs))
        for numba,pair in enumerate(possible_stitch_pairs):
            #print("pair = " + str(pair))
            sizes = [face_0_facet_sizes[pair[0]],face_1_facet_sizes[pair[1]]]
            min_area = min(sizes)
            max_area = max(sizes)

            ratio = min_area/max_area

            #print(f"ratio = {ratio}")
            #print(f"Total size  = {min_area + max_area}")
            if ratio >= size_ratio_threshold:
                face_pair_sizes[numba] = min_area + max_area
                face_size_ratios[numba] = ratio
            
                #print(f"face_pair_sizes[numba] = {face_pair_sizes[numba]}, face_size_ratios[numba] = {face_size_ratios[numba]}")


        #check that made it past stitch ratio threshold

        #best possible stitch pair is just the maximum sized matching ones
        best_index = np.where(face_pair_sizes == max(face_pair_sizes))
        best_stitch_pair = possible_stitch_pairs[best_index][0]
        best_stitch_pair_size = face_pair_sizes[best_index][0]
        best_stitch_pair_size_ratio = face_size_ratios[best_index][0]
        
        #get the distance of the best_stitch_pair
        best_stitch_pair_distance = a_b_distance[possible_stitch_pairs[best_index][0][0],
                                                   possible_stitch_pairs[best_index][0][1]]
        
        print("best_stitch_pair = " + str(best_stitch_pair))
        print("best_stitch_pair_size = " + str(best_stitch_pair_size))
        print("best_stitch_pair_distance = " + str(best_stitch_pair_distance))
        print("best_stitch_pair_size_ratio = " + str(best_stitch_pair_size_ratio))

        child_meshes_stitch_facets[i] = [best_stitch_pair[0],best_stitch_pair[1]]
        child_meshes_stitch_face_ratios[i] = best_stitch_pair_size_ratio
        child_meshes_stitch_distances[i] = best_stitch_pair_distance
        
#         if i == 1:
#             print("Just processed piece 1")
#             raise Exception("stoping")

    
#     if 1 in child_meshes_stitch_facets.keys():
#         print("Just processed piece 1")
#         raise Exception("stoping")
    
    
    #if there were no possible stitch points found
    if len(child_meshes_stitch_facets.keys()) == 0:
        #increment the no children flag multiplier
        
        no_new_children_multiplier += 1
        print(f"no stitch points found IN ALL CHILDREN --> relaxing the parameters time {no_new_children_multiplier}")
        continue
        
    # makes sure that no two child branches try to connect to the same main branch
    from collections import Counter
    mesh_stitch_counter = Counter(np.array([val for val in child_meshes_stitch_facets.values()])[:,1])

    repeat_main_facets = [key for key,val in mesh_stitch_counter.items() if (key != -1 and val > 1)] #gets the main mesh facet with multiples
    print("repeat_main_facets = " + str(repeat_main_facets))


    #how to fix that some faces are trying to branch to same main facet
    #make it iterate through all of the repeats
    if len(repeat_main_facets)>0:
        for repeat_main in repeat_main_facets:
            child_mesh_double_indexes = [key for key,val in child_meshes_stitch_facets.items() if val[1] == repeat_main]
            print("child_mesh_double_indexes = " + str(child_mesh_double_indexes))


            #decide which one to keep and then ditch all the rest --> pick the CLOSEST ONE, and not the best matching area:

            ###### picks the closest area
            min_distance = 10000000000
            min_child = -1


            for child_index in child_mesh_double_indexes:
                current_distance = child_meshes_stitch_distances[child_index]

                if current_distance < min_distance:
                    min_child = child_index
                    min_distance = current_distance

            print(f"min_child = {min_child}, max_ratio = {min_distance}")

    #         ###### picks the maximum area
    #         max_ratio = -1
    #         max_child = -1


    #         for child_index in child_mesh_double_indexes:
    #             current_ratio = child_meshes_stitch_face_ratios[child_index]

    #             if current_ratio > max_ratio:
    #                 max_child = child_index
    #                 max_ratio = current_ratio

    #         print(f"max_child = {max_child}, max_ratio = {max_ratio}")



            #remove the others from the stitch facets
            for double_index in child_mesh_double_indexes:
                if double_index != min_child:
                    del child_meshes_stitch_facets[double_index]
    
    """
    Pseudocode for stitching:
    1) For each pair in the child_meshes_stitch_facets:
    a. Get the child mesh for that pair
    b. Get the list of faces for the child facet (from the facet number)
    c. Get the list of faces for the main facet (from the main number)

    d. Get the original number of faces and vertices in the main mesh
    d2. Use the orignal number of faces and add to list of faces for child facet to offset them correctly
        - Save this number list in a dictionary (to use for later and creating the submesh)
    e. Add the two meshes together to get big mesh
    f. Send the two meshes and the two facet lists to the restitching function to get a main mesh that is stitched up
     - but send it to function that doesnt delete the original facet faces 
         (because this would remove meshes from original and screw up facet number)
    g. reassign the main_mesh to this newly stitched up mesh
    h. recompute the facets for the main mesh



    """
    

    current_main_mesh = main_mesh.copy()
    
    main_mesh_facet_index_to_delete = []

    
    #if there were no possible stitch points found
    if len(child_meshes_stitch_facets.keys()) == 0:
        #increment the no children flag multiplier
        
        no_new_children_multiplier += 1
        print(f"no stitch points found IN ALL CHILDREN --> relaxing the parameters time {no_new_children_multiplier}")
        continue
    else: #if there were stitch points found
        print("child_meshes_stitch_facets = " + str(child_meshes_stitch_facets))
        for child_key,pair in child_meshes_stitch_facets.items():
            """
            child_key has the child that is currently being processed
            
            pair has the facet ids that are to be stitched together
            
            """
            
            child_used_facet_index = pair[0]
            main_used_faceet_index = pair[1]
            
            stitch_time = time.time()
            print(f"---Stitching child {child_key} with pair: {pair}---")
            current_child_mesh = child_meshes[child_key]
            current_child_facet_faces = child_meshes_facets[child_key][0][pair[0]]


            current_main_mesh_facet_faces = main_mesh_facets[pair[1]]

            #Get the original number of faces and vertices in the main mesh
            original_mesh_faces_len = len(current_main_mesh.faces)
            current_child_facet_faces_adjusted = current_child_facet_faces + original_mesh_faces_len

            #Save the faces number for deletion later
            child_faces_to_remove += current_child_facet_faces_adjusted.tolist()
            main_faces_to_remove += current_main_mesh_facet_faces.tolist()

            combined_mesh = current_main_mesh + current_child_mesh

            #how to stitch up the mesh
            start_time = time.time()
            current_main_mesh = stitch_mesh_piece_vp4(new_mesh=combined_mesh,
                                                           facet_1=current_main_mesh_facet_faces,
                                                           facet_2=current_child_facet_faces_adjusted,
                                                          delete_facets=False,
                                                          return_added_mesh=False,
                                                           fix_normals = False)
            
            print(f"returned from stitch mesh: {time.time() - start_time}")
            
            
            #Don't need to do any deletion now 
            
            #add the child to processed child
            children_processed.append(child_key)
            print(f"Finished stitching child {child_key} : {time.time() - stitch_time}")
            
            """
            Add all of the child facets that weren't the ones used to the main mesh facets (with adjusted facet numbers)
            """
            original_mesh_faces_len #the original number of faces in the main mesh (including all those stitched before)
            
            #remove certian rows from array of the child facets and cetners
            current_child_facet_faces_with_deletion = np.delete(child_meshes_facets[child_key][0],(child_used_facet_index),axis=0)
            current_child_facet_centers_with_deletion = np.delete(child_meshes_facets[child_key][1],(child_used_facet_index),axis=0)

            #have to adjust the faces of the child_facets list to account for the faces off set
            current_child_facet_faces_with_deletion = current_child_facet_faces_with_deletion + original_mesh_faces_len

            #append this list to the main mesh list
            main_mesh_facets = np.concatenate([main_mesh_facets,current_child_facet_faces_with_deletion])
            main_mesh_facets_centers = np.concatenate([main_mesh_facets_centers,current_child_facet_centers_with_deletion])

            #save off the facets to delete for the main mesh at the end of the loop
            main_mesh_facet_index_to_delete.append(pair[1])
            



        #reset the no_new_children_multiplier because there were successful stitching
        no_new_children_multiplier = 0
        """recompute the main mesh facets  ###Not going to recompute anymore
        #main_mesh_facets,main_mesh_facets_centers = filter_final_facets(main_mesh)
        
        Now we just remove the main mesh facets that are used and reassign
        """
        main_mesh = current_main_mesh
    
    
        #at the end of the big loop have to delete the facets used from main mesh
        #remove certian rows from array
        main_mesh_facets = np.delete(main_mesh_facets,(main_mesh_facet_index_to_delete),axis=0)
        main_mesh_facets_centers = np.delete(main_mesh_facets_centers,(main_mesh_facet_index_to_delete),axis=0)

        
        print(f"***************Finished big stitching iteration: {time.time() - stitch_loop_time}***************")
        
#         if 0 in children_processed and 1 in children_processed:
#             print("Exiting before does piece 2")
#             break

        if len(np.unique(children_processed)) == len(child_meshes):
            print("All children have been processed")
            break
    


Starting Child 0
699 11118
Done distance matrix: 0.22433257102966309
Child 0 There were no points close enough
Starting Child 1
644 11118
Done distance matrix: 0.22945070266723633
Done Generating final pairs: 0.23956942558288574
best_stitch_pair = [ 297 2053]
best_stitch_pair_size = 126407.37080777145
best_stitch_pair_distance = 322.83217480906654
best_stitch_pair_size_ratio = 0.4329072487180667
Starting Child 2
485 11118
Done distance matrix: 0.17730927467346191
Done Generating final pairs: 0.25577569007873535
best_stitch_pair = [  11 7052]
best_stitch_pair_size = 3722036.954014628
best_stitch_pair_distance = 134.73286075380463
best_stitch_pair_size_ratio = 0.936193249947911
Starting Child 3
887 11118
Done distance matrix: 0.28705525398254395
Child 3 There were no points close enough
Starting Child 4
521 11118
Done distance matrix: 0.16857337951660156
Child 4 There were no points close enough
Starting Child 5
203 11118
Done distance matrix: 0.06521773338317871
Child 5 There were no po

Exception: Finished the mesh stitiching

In [ ]:
"""
There is a potential problem where delete facet from facet list but 
there are facets that are overlapping on the old way of calculating facets
which is used for the big pieces --> 

So potential problem where could have connecting to same mesh faces because 
multiple facets for that main mesh face exist

Possible solutions: 
1) Make the way of generating them check for overlaps and resolve
2) Add a face DO NOT CONNNECT LIST that checks if any faces currently processed
have been connected to already --> not great solution
3) Just leave it be and may create some self intersecting faces

"""

In [26]:
len(child_faces_to_remove),len(main_faces_to_remove)

(837, 739)

In [27]:
#now to extract the full mesh after it's done so don't screw up all the facets numbers:


#remove all of the processed facets from the main mesh
#now take away the original facet faces:
total_faces = np.linspace(0,len(main_mesh.faces)-1,len(main_mesh.faces)).astype("int")


#these are the faces that need to be removed
facet_faces = np.hstack([child_faces_to_remove ,main_faces_to_remove])
faces_to_keep = set(total_faces).difference(set(facet_faces))


main_mesh = main_mesh.submesh([list(faces_to_keep)])[0]


In [28]:
from print_trimesh import print_trimesh

print_trimesh(main_mesh,"./test_meshes/entire_neuron_facets_optimized.off")

In [29]:
print(f"Total time for restitching = {time.time() - restitch_time}")

Total time for restitching = 183.9891858100891


In [31]:
#try and compress the file:
import bz2
from shutil import copyfileobj

with open('./test_meshes/entire_neuron_facets_optimized.off', 'rb') as input:
    with bz2.BZ2File('./test_meshes/entire_neuron_facets_optimized.off.bz2', 'wb', compresslevel=9) as output:
        copyfileobj(input, output)

In [38]:

filepath = './test_meshes/entire_neuron_facets_optimized.off.bz2'
newfilepath = './test_meshes/entire_neuron_facets_optimized.off.bz2.off'

with open(newfilepath, 'wb') as new_file, bz2.BZ2File(filepath, 'rb') as file:
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(data)

In [34]:
import os
num = os.path.getsize('./test_meshes/entire_neuron_facets_optimized.off.bz2')
num/1000/1000



26.149690999999997

In [39]:
#check that it can be loaded into trimesh so uncompressed right
reloaded_mesh = trimesh.load_mesh("./test_meshes/entire_neuron_facets_optimized.off.bz2.off")




In [ ]:
"""
Conclusion after observations: 
1) For some of neurons connecting from very far away (need to reduce size)
2) Seems like it is working just as good as other neuron stitcher i had
3) the stitching doesn't always work the best but it should do for now

"""

"""
Things still need to do:
1) Investigate those that aren't getting added and have reason why not
2) Make sure the normals can be flipped (and better if can just fix normals as make them)
"""

In [ ]:
#main_mesh_fixed = trimesh.fix_normals(main_mesh)

# Exploring How to Transfer Labels

In [ ]:
# """
# Copies to practice on

# """

In [ ]:
# #copying the data to be used in the test
# child_meshes_facets_test= deepcopy(child_meshes_facets)
# pair_test = deepcopy(pair)

# main_mesh_facets_test = deepcopy(main_mesh_facets)
# main_mesh_facets_centers_test = deepcopy(main_mesh_facets_centers)

In [ ]:
# """ Child data """
# child_key #the current child we are processing
# pair #has the child mesh and the main mesh facet index
# current_child_facet_faces = child_meshes_facets_test[child_key][0] #all of the child facets 
# current_child_facet_centers = child_meshes_facets_test[child_key][1]
# child_used_facet_index = pair_test[0]
# child_used_facet_faces = current_child_facet_faces[child_used_facet_index] #current facet that is being used
# child_used_facet_centers = current_child_facet_centers[child_used_facet_index] #current facet that is being used

# """ Main mesh data """
# main_used_faceet_index = pair[1]

# main_used_facet_faces = main_mesh_facets_test[main_used_faceet_index]
# main_used_facet_centers = main_mesh_facets_centers_test[main_used_faceet_index]

# #checking right now
# child_used_facet_centers

In [ ]:
# """ Check how to apply the operations
# """

# #make a copy
# current_child_facet_faces_copy = current_child_facet_faces.copy()
# current_child_facet_centers_copy = current_child_facet_centers.copy()
# print("before deletion facets len = " + str(len(current_child_facet_faces_copy)) + ", centers len = " + str(len(current_child_facet_centers_copy)))

# print("NO COPIES before deletion facets len = " + str(len(current_child_facet_faces)) + ", centers len = " + str(len(current_child_facet_centers)))

# #delete a certain row from array
# current_child_facet_faces_copy_with_deletion = np.delete(current_child_facet_faces_copy,(child_used_facet_index),axis=0)
# current_child_facet_centers_copy_with_deletion = np.delete(current_child_facet_centers_copy,(child_used_facet_index),axis=0)

# #delete without making the copies

# current_child_facet_faces_with_deletion = np.delete(current_child_facet_faces,(child_used_facet_index),axis=0)
# current_child_facet_centers_with_deletion = np.delete(current_child_facet_centers,(child_used_facet_index),axis=0)

# print("AFTER deletion facets len = " + str(len(current_child_facet_faces_copy_with_deletion)) + ", centers len = " + str(len(current_child_facet_centers_copy_with_deletion)))

# print("NO COPIES AFTER deletion facets len = " + str(len(current_child_facet_faces_with_deletion)) + ", centers len = " + str(len(current_child_facet_centers_with_deletion)))

# print("NO COPIES AFTER deletion ORIGINALS facets len = " + str(len(current_child_facet_faces)) + ", centers len = " + str(len(current_child_facet_centers)))


# """
# CONCLUSION: so don't need to make copies and remains unchanged

# """
# current_child_facet_faces_with_deletion[0] = [None]
# print(current_child_facet_faces_with_deletion[0])
# print(current_child_facet_faces[0])

# """
# CONCLUSION: so don't need to make copies and remains unchanged and altering doesnt affect it

# """

In [ ]:
# #remove certian rows from array
# current_child_facet_faces_with_deletion = np.delete(current_child_facet_faces,(child_used_facet_index),axis=0)
# current_child_facet_centers_with_deletion = np.delete(current_child_facet_centers,(child_used_facet_index),axis=0)

# ##### DON'T WANT TO DELETE THE MAIN MESH FACETS JUST YET ####

# #have to adjust the faces of the child_facets list to account for the faces off set
# current_child_facet_faces_with_deletion = current_child_facet_faces_with_deletion + original_mesh_faces_len

# #append this list to the main mesh list
# main_mesh_facets_test_new = np.concatenate([main_mesh_facets_test,current_child_facet_faces_with_deletion])
# main_mesh_centers_test_new = np.concatenate([main_mesh_facets_centers_test,current_child_facet_centers_with_deletion])

# print(len(main_mesh_facets_test) + len(current_child_facet_faces) -1 ,len(main_mesh_facets_test_new))
# print(len(main_mesh_facets_centers_test) + len(current_child_facet_centers) -1 ,len(main_mesh_centers_test_new))

# #save off the facets to delete for the main mesh at the end of the loop
# main_mesh_facet_index_to_delete = []
# main_mesh_facet_index_to_delete.append(pair[1])
# main_mesh_facet_index_to_delete.append(0)


# #at the end of the big loop have to delete the facets used from main mesh
# #remove certian rows from array
# main_mesh_facets_test_new_with_deletion = np.delete(main_mesh_facets_test_new,(main_mesh_facet_index_to_delete),axis=0)
# main_mesh_centers_test_new_with_deletion = np.delete(main_mesh_centers_test_new,(main_mesh_facet_index_to_delete),axis=0)

# print(len(main_mesh_facets_test_new),len(main_mesh_facets_test_new_with_deletion))
# print(len(main_mesh_centers_test_new),len(main_mesh_centers_test_new_with_deletion))


In [ ]:
# len(main_mesh_facets_test_new),len(main_mesh_facets_test_new_with_deletion)

In [ ]:
# original_mesh_faces_len

In [ ]:
# current_child_facet_faces_with_deletion[0]

In [ ]:
# (current_child_facet_faces_with_deletion + original_mesh_faces_len)[0] - current_child_facet_faces_with_deletion[0]

In [ ]:
# # EXPORT THE MAIN MESH
# """
# Pseudocode for mapping labels
# 1) Get the child index and the pair of child facet and main facet to stitch
# 2) Create combined mesh (that just adds faces on top of each other)
#     main facet faces to remove 
# 3) Adjusts the child facets so now they correspond to the combined mesh face numbers
# 4) Add the current_child_facet_faces_adjusted and current_main_mesh_facet_faces to the list to delete
# -- will know which faces to delete at the end (currently doing this)

# How to transfer the labels: SO DON'T HAVE TO RECOMPUTE
# 1) at the very end of each stitch loop need to add the children facets and their faces list to the main facets list (this won't interfere with any stitching)
#     but don't add the one that was just used 
#     --> have to add the same center points as well (based on the )
# 2) At the very end have to delete the main mesh facets that were used and their center points

# --> then rerun the loop and everything should run fine with the new main mesh facets being different
# """